In [64]:
import scipy as sp
import numpy as np
import scipy.sparse.linalg
import scipy.sparse
import networkx as nx
from networkx.algorithms.link_analysis.pagerank_alg import google_matrix
from sklearn.preprocessing import normalize
from random import sample
import random
import operator


In [65]:
G = nx.read_edgelist('./Networks/cit-HepTh.txt', comments='#',
                     create_using=nx.DiGraph(), 
                     delimiter='\t', 
                     nodetype=int, 
                     encoding='utf-8')


# Creating the base set

We create a function that constructs a random set of 2000 nodes, as a root set, and then extends it to create the base set for our algorithms.
The extension is based on the edges and the connection of two nodes.
If the base set nodes are not connected, we keep the largest connected component

In [67]:
def nodeSet(G):
    '''
    A function that creates a random set of 2000 nodes, as a root set, and then extends it to create the base set for our algorithms.
    If the base set nodes are not connected, we keep the largest connected component
    :param G: The initial graph.
    :returns H: The base set of nodes, from our initial graph.
    '''
    sampleN = sample(G.nodes(), 2000)
    edges=list(G.out_edges(sampleN))
    in_edges=G.in_edges(sampleN)
    edges.extend(in_edges)
    edgesG=nx.DiGraph(edges)
    newSampleNodes=edgesG.nodes()
    H=G.subgraph(newSampleNodes)
    if nx.is_connected(nx.Graph(H))==False:
        largest_cc = max(nx.connected_components(nx.Graph(H)), key=len)
        H=H.subgraph(largest_cc)
    
    return H

In [68]:
graph=nodeSet(G)
print(len(graph.nodes()))

17442


# Salsa algorithm

The SALSA algorithm (Stochastic Approach for Link-Structure Analysis) is an algorithm inspired by PageRank and HITS. More precisely SALSA is a modification of HITS algorithm, that conducts a random walk (an attribute of PageRank) between two graphs. The two graphs are created by taking the graph g and looking at it as a bipartite graph. On the one side the nodes represent hubs and on the other side authorities. We use the max iter parameter as 600, to let the algorithm converge.

In [69]:
def salsa(G, max_iter=600, tol=1.0e-8, nstart=None, normalized=True):
    '''
    SALSA implementation.
    :param G: the Graph G that we want to find the SALSA hits and authorities.
    :param max_iter: maximum number of iterations before the algorithm needs to converge. Else break.
    :param tol: tolerance of permited error til convergence.
    '''
    A = nx.adjacency_matrix(G, nodelist=list(G)).astype('float')
    (rows, cols) = A.shape 
    colNormA=normalize(A, norm='l1', axis=0)
    rowNormA=normalize(A, norm='l1', axis=1)
    
    authM = rowNormA.T * colNormA 
    a = np.ones((rows, 1)) / rows  # authority vector
    
    i = 0
    while True:
        flag = a
        a = authM * a
        a = a / a.max()
        error = np.absolute(a - flag).sum()
        if error < tol:
            break
        if i > max_iter:
            print("BREAK")
            print(i)
            break
            #raise nx.PowerIterationFailedConvergence(max_iter) --> Doesn't work, apparently....

        i += 1
    a = np.asarray(a).flatten()
    h = np.asarray(colNormA * a).flatten()  # h=adjacency_matrix_c*a
    if normalized:
        #print(dict(G.in_degree()))
        h = h / sum(dict(G.in_degree()).values())
        a = a / sum(dict(G.out_degree()).values())
    
    hubs = dict(zip(G, map(float, h)))
    authorities = dict(zip(G, map(float, a)))
    
    return hubs, authorities
        
    

# Checking the overlaps 

We create a function that checks the overlap percentages between the needed set of hubs, authorities and PageRank scores. 

In [70]:
'''def checkOverlaps(sA,sH,hA,hH,pr):
    overlapSASH=[]
    overlapHAHH=[]
    overlapPRHA=[]
    overlapPRSA=[]
    overlapHASA=[]
    overlapSHHH=[]
    
    
    for i in range(1,21):
        c1=c2=c3=c4=c5=c6=0
        for j in range(1,i):
            if list(sA.keys())[j]==list(sH.keys())[j]:
                c1 += 1
            if list(hA.keys())[j]==list(hH.keys())[j]:
                c2 += 1
            if list(pr.keys())[j]==list(hA.keys())[j]:
                c3 += 1
            if list(pr.keys())[j]==list(sA.keys())[j]:
                c4 += 1
            if list(hA.keys())[j]==list(sA.keys())[j]:
                c5 += 1
            if list(sH.keys())[j]==list(hH.keys())[j]:
                c6 += 1
        

        
        overlapSASH.append(c1)
        overlapHAHH.append(c2)
        overlapPRHA.append(c3)
        overlapPRSA.append(c4)
        overlapHASA.append(c5)
        overlapSHHH.append(c6)
        

    
    return sum(overlapSASH)/20, sum(overlapHAHH)/20, sum(overlapPRHA)/20, sum(overlapPRSA)/20, sum(overlapHASA)/20, sum(overlapSHHH)/20
        
        

        '''

'def checkOverlaps(sA,sH,hA,hH,pr):\n    overlapSASH=[]\n    overlapHAHH=[]\n    overlapPRHA=[]\n    overlapPRSA=[]\n    overlapHASA=[]\n    overlapSHHH=[]\n    \n    \n    for i in range(1,21):\n        c1=c2=c3=c4=c5=c6=0\n        for j in range(1,i):\n            if list(sA.keys())[j]==list(sH.keys())[j]:\n                c1 += 1\n            if list(hA.keys())[j]==list(hH.keys())[j]:\n                c2 += 1\n            if list(pr.keys())[j]==list(hA.keys())[j]:\n                c3 += 1\n            if list(pr.keys())[j]==list(sA.keys())[j]:\n                c4 += 1\n            if list(hA.keys())[j]==list(sA.keys())[j]:\n                c5 += 1\n            if list(sH.keys())[j]==list(hH.keys())[j]:\n                c6 += 1\n        \n\n        \n        overlapSASH.append(c1)\n        overlapHAHH.append(c2)\n        overlapPRHA.append(c3)\n        overlapPRSA.append(c4)\n        overlapHASA.append(c5)\n        overlapSHHH.append(c6)\n        \n\n    \n    return sum(overlapSASH)

In [71]:
def checkOverlaps(sA,sH,hA,hH,pr):
    '''
    Checks overlaps of the first 20 nodes by creating two sets from lists, repetitively, and checking the intersection.
    :param sA: SALSA Authority scores
    :param sH: SALSA Hub scores
    :param hA: HITS Authority scores
    :param hH: HITS Hub scores 
    :param pr: PageRank scores
    :returns: The average of the percentages of each overlap
    '''
    overlapSASH=[]
    overlapHAHH=[]
    overlapPRHA=[]
    overlapPRSA=[]
    overlapHASA=[]
    overlapSHHH=[]
    
    
    for i in range(1,21):
        overlapSASH.append((len(list(set(list(sA.keys())[:i])&set(list(sH.keys())[:i]))))/i)
        overlapHAHH.append((len(list(set(list(hA.keys())[:i])&set(list(hH.keys())[:i]))))/i)
        overlapPRHA.append((len(list(set(list(pr.keys())[:i])&set(list(hA.keys())[:i]))))/i)
        overlapPRSA.append((len(list(set(list(pr.keys())[:i])&set(list(sA.keys())[:i]))))/i)
        overlapHASA.append((len(list(set(list(hA.keys())[:i])&set(list(sA.keys())[:i]))))/i)
        overlapSHHH.append((len(list(set(list(sH.keys())[:i])&set(list(hH.keys())[:i]))))/i)
        

    
    return sum(overlapSASH)/20, sum(overlapHAHH)/20, sum(overlapPRHA)/20, sum(overlapPRSA)/20, sum(overlapHASA)/20, sum(overlapSHHH)/20
        
        


In [72]:
overlapSASHT=[]
overlapHAHHT=[]
overlapPRHAT=[]
overlapPRSAT=[]
overlapHASAT=[]
overlapSHHHT=[]
    

for i in range(10):
    
    graph=nodeSet(G)
    
    ######SALSA########
    h,a=salsa(graph)
    sorted_tuples=(sorted(a.items(), key=operator.itemgetter(1),reverse=True))
    a = {k: v for k, v in sorted_tuples}
    sorted_tuples=(sorted(h.items(), key=operator.itemgetter(1),reverse=True))
    h = {k: v for k, v in sorted_tuples}    
    
    ######HITS#########
    
    hits_h,hits_a=nx.hits(graph,max_iter=600)
    sorted_tuples=(sorted(hits_a.items(), key=operator.itemgetter(1),reverse=True))
    hits_a = {k: v for k, v in sorted_tuples}
    sorted_tuples=(sorted(hits_h.items(), key=operator.itemgetter(1),reverse=True))
    hits_h = {k: v for k, v in sorted_tuples}
    
    #######PAGERANK#######
    
    pr = nx.pagerank(graph, alpha=0.9,max_iter=600)
    sorted_tuples=(sorted(pr.items(), key=operator.itemgetter(1),reverse=True))
    pr = {k: v for k, v in sorted_tuples}

    #######OVERLAP########
    print(i)
    print(list(a.keys())[:20])
    print(list(h.keys())[:20])
    print(list(hits_a.keys())[:20])
    print(list(hits_h.keys())[:20])
    print(list(pr.keys())[:20])
    
    
    overlapSASH, overlapHAHH, overlapPRHA, overlapPRSA, overlapHASA, overlapSHHH=checkOverlaps(a,h,hits_a,hits_h,pr)
    
    overlapSASHT.append(overlapSASH)
    overlapHAHHT.append(overlapHAHH)
    overlapPRHAT.append(overlapPRHA)
    overlapPRSAT.append(overlapPRSA)
    overlapHASAT.append(overlapHASA)
    overlapSHHHT.append(overlapSHHH)

BREAK
601
0
[9711200, 9802150, 9802109, 9407087, 9510017, 9610043, 9503124, 9908142, 9906064, 9408099, 9510135, 9711162, 9510209, 9905111, 9410167, 9611050, 9601029, 9602022, 9603167, 9603142]
[9905111, 9710046, 110055, 210157, 7170, 101126, 204089, 109162, 9802067, 207116, 201253, 9809039, 9802051, 9712253, 111208, 210292, 11197, 209230, 209241, 106048]
[9711200, 9802150, 9802109, 9510017, 9905111, 9503124, 9610043, 9510135, 9803131, 9410167, 9611050, 9407087, 9601029, 9510209, 9906064, 9408099, 9805114, 9803002, 9908142, 9409089]
[9905111, 110055, 7170, 210157, 101126, 9710046, 9811019, 9806199, 9802051, 9912164, 9908004, 9903268, 201253, 9139, 9712253, 3136, 9901050, 9809039, 9807171, 9901018]
[9407087, 9503124, 9207016, 9510017, 9402044, 9201015, 9410167, 9402002, 9711200, 9205027, 9408099, 9304154, 9510135, 9207053, 9205068, 9610043, 9401139, 9305185, 9504090, 208020]
1
[9711200, 9802150, 9802109, 9407087, 9510017, 9610043, 9503124, 9908142, 9408099, 9906064, 9905111, 9510209, 951

In [73]:
print("Overlaps between SALSA authorities and SALSA Hubs:  ",overlapSASHT)
print("The average:  ",sum(overlapSASHT)/10)
print()
print("Overlaps between HITS authorities and HITS Hubs:  ",overlapHAHHT)
print("The average:  ",sum(overlapHAHHT)/10)
print()
print("Overlaps between PageRank and HITS Authorities:  ",overlapPRHAT)
print("The average:  ",sum(overlapPRHAT)/10)
print()
print("Overlaps between PageRank and SALSA Authorities:  ",overlapPRSAT)
print("The average:  ",sum(overlapPRSAT)/10)
print()
print("Overlaps between SALSA authorities and HITS Authorities:  ",overlapHASAT)
print("The average:  ",sum(overlapHASAT)/10)
print()
print("Overlaps between SALSA Hubs and HITS Hubs:  ",overlapSHHHT)
print("The average:  ",sum(overlapSHHHT)/10)

Overlaps between SALSA authorities and SALSA Hubs:   [0.020880295100496337, 0.0334385701587714, 0.020880295100496337, 0.0334385701587714, 0.028893115613316856, 0.024726448946650187, 0.017308866529067764, 0.028893115613316856, 0.020880295100496337, 0.0334385701587714]
The average:   0.02627781424801549

Overlaps between HITS authorities and HITS Hubs:   [0.07572031619051742, 0.07572031619051742, 0.07572031619051742, 0.08822031619051743, 0.08822031619051743, 0.07572031619051742, 0.06572031619051744, 0.07572031619051742, 0.0573869828571841, 0.08822031619051743]
The average:   0.0766369828571841

Overlaps between PageRank and HITS Authorities:   [0.3179439592055691, 0.30683770434931423, 0.3466331866447966, 0.3122810804176903, 0.30874466585186405, 0.3186005248621348, 0.3705599543215643, 0.3068755831371931, 0.33818205444366434, 0.2690508435922523]
The average:   0.3195709556826043

Overlaps between PageRank and SALSA Authorities:   [0.35477115978276974, 0.3458158789524889, 0.3751054088670188